# Microtask-1

> Produce a notebook showing (and producing) a list with the activity per quarter: number of new committers, submitters of issues, and submitters of pull/merge requests, number of items (commits, issues, pull/merge requests), number of repositories with new items (all of this per quarter) as a table and as a CSV file using plain python3 (no pandas).


I am using the same data source file which is used in the [microtask-0](https://github.com/vchrombie/chaoss-microtasks/blob/master/microtask-0/microtask-0.ipynb) i.e, [elasticsearch-py](https://github.com/elastic/elasticsearch-py) project which is located in the `data/` folder of the repository.

## Importing the neccessary modules

In [1]:
# json library is used to handle json files, here, it is the data source 
# retrieved by the perceval module.
import json 
# to write and read csv files, to show the output in the end
import csv  

# to handle the time formats, like to determine 'created_at' of an issue or pr.
from datetime import datetime 
# dictionaries are a convenient way to store data for later retrieval by name (key).
from collections import defaultdict  

# it is used to send http requests, I used this to get the year in which the project 
# created to do the analysis, using requests and github api.
import requests 
# open source python module to pretty print a csv file. 
# ref: https://github.com/jazzband/prettytable
from prettytable import from_csv

## Functions to return the details of the contribution types

_Commit_ has a different json structure when compared to *issue* and *pull_request*.

In [3]:
# commit has a different json structure unlike issue/pr

def details_commit(commit):
    """
    Get the contents of the commit.
    
    This method gives, by taking the line data, 
    the summary of the commit.
    
    :param item: line json data of the commit
    :return: content of the line
    """
    # load the commit data into the object
    data = commit['data']
    # traverse through the json line to find the required data
    content ={
            # get the hash of the commit
            'hash': data['commit'],
            # get the author_name
            'author': data['Author'],  
            # get the date at which the commit was created
            'created_date': datetime.strptime(data['CommitDate'],
                                              "%a %b %d %H:%M:%S %Y %z")  
    }
    # return the content
    return content

In [4]:
# as issue/pr has the same json structure in the data source scraped by perceval
# I wrote a single function to get the either issue/pr details 

def details_ipr(item):
    """
    Get the contents of the issue/pr.
    
    This method gives, by taking the line data, 
    the summary of the issue/pr.
    
    :param item: line json data of the issue/pr
    :return: content of the line
    """
    # load the commit data into the object
    data = item['data']
    # traverse through the json line to find the required data
    content ={
            # get the hash of the issue/pr
            'hash': data['id'],
            # get the author_name
            'author': data['user']['login'],  
            # get the date at which the issue/pr was created
            'created_date': datetime.strptime(data['created_at'],"%Y-%m-%dT%H:%M:%SZ")  
    }
    # return the content
    return content 

## Dividing the data source into contribution types

In [5]:
def get_contents(repo):
    """
    Get the contents of the project.
    
    This method gives, by taking the data retrived by perceval, 
    the content of the repository.
    
    :param repo: get the name of the repository
    
    :return: contents of the repository
    """
    # contents is to store the details of each contribution whether it is a commit, issue or pr.
    # using a defaultdict of list so that I can store the sorted details according to the ctype as (key, value) 
    contents = defaultdict(list)

    # to filter out commit, issue, pr details from the data source and store them seperately in dict.
    # loading the file into an object
    with open('../data/%s.json'%repo) as datasrc:
        for line in datasrc:
            # load the line in the json format so as to iterate to get the required results
            line = json.loads(line)
            # if it is a commit, get the details of commit
            if line['category'] == 'commit':    
                content = details_commit(line) 
            # if it is a issue, get the details of issue
            elif line['category'] == 'issue':    
                content = details_ipr(line)
            # if it is a pr, get the details of pr
            elif line['category'] == 'pull_request':    
                content = details_ipr(line) 
            # add the (key, value) to the list
            contents[line['category']].append(content)
    # return the contents
    return contents

## Defining Quaters

The calendar year can be divided into four quarters, often abbreviated as Q1, Q2, Q3, and Q4.
- First quarter, Q1: 1 January – 31 March (90 days or 91 days in leap years)
- Second quarter, Q2: 1 April – 30 June (91 days)
- Third quarter, Q3: 1 July – 30 September (92 days)
- Fourth quarter, Q4: 1 October – 31 December (92 days)

Reference: https://en.wikipedia.org/wiki/Calendar_year

Each Quater is represented as **Qi yyyy** where *i* is the quater number and _yyyy_ is the year.

In [2]:
def define_quarters():
    """
    Define the quaters of the year.
    
    :return: quaters tuple of the year
    """
    # as said the year can be divided into four different quaters 
    # so I divided the year into 4 quaters according to the information mentioned above.
    QUARTERS = (
        ({'month':1,'day':1},  {'month':3,'day':31}),
        ({'month':4,'day':1},  {'month':6,'day':30}),
        ({'month':7,'day':1},  {'month':9,'day':30}),
        ({'month':10,'day':1}, {'month':12,'day':31}),
    )
    # return the quarters tuple
    return QUARTERS

## Splitting the data quaterly

In [6]:
# I have to write this function to get the project created_date and present_date.

# I found it difficult to scrape through the data retrieved by the perceval and find the
# dates so I had to find them through this hack

def get_dates(repo):
    """
    Get the project created date and present date.
    
    This method gives, by sending a request to github api, 
    and gets the project created date and present date.
    
    :param repo: get the name of the repository
    :return 
    """
    # using github api and requests, the data of the repository is stored in the object
    repodata =json.loads(requests.get("https://api.github.com/repos/elastic/%s"%(repo)).text)
    # currently, this works only for elastic organization as I have hard-coded the orgranization name
    # in the URL but passed the repository name as the parameter
    # This can be solved by passing the org name in the main function too as "org/repo" [TO-DO]

    # using datetime library, got the year in which the project was created and the current year
    created =datetime.strptime(repodata['created_at'][:10], "%Y-%m-%d").year
    present =datetime.strptime(repodata['updated_at'][:10], "%Y-%m-%d").year
    # return the dates
    return created, present

In [7]:
# funtion to divide the contributions into quaterly based on the QUATERS

# the first_year and last_year is divided into for quaters
def quarterwise(first_year,last_year):
    """
    Divides the contributions quaterly based on QUATERS.
    
    The method divides, from a data source, the contributions 
    in the repository quaterly between the first and last year.
        
    :param first_year: year when the project started
    :param last_year: year when the project ended
    
    :returns: current year of the quater
    :returns: quarter of the year
    :returns: start date of the quater
    :returns: end date of the quater
    """
    QUARTERS = define_quarters()
    # iterating through the years, from staring to ending
    for year in range(first_year, last_year+1):
        # iterating through the enumerated quaters
        for quarter,(start,end) in enumerate(QUARTERS):
            # returning the quaters start and end time period
            start = datetime(year,**start)
            end = datetime(year,**end)
            # yield is used to return multiple files in one particular order
            yield year,quarter,start,end  

In [8]:
def summerize_quarterwise(repo):
    """
    Divides the contributions quaterly based on QUATERS.
    
    The method summerizes , from a data source, the contributions 
    in the repository quaterly of the repository.
    
    :param repo: get the name of the repository to be analyzed
    
    :returns: quarters of the year
    :returns: activities list which has the total activity of the repo
    :returns: new contributor list 
    """
    contents =get_contents("%s"%repo)
    
    # there are three types of contributions - commit, issue, and pr
    # so, I created a tuple which has the contribution types
    ctypes = ('commit','pull_request','issue')

    # initializing a empty quaters list to store the quaters of the project
    quarters = []

    # using again a defaultdict(list) to store the activites and it's vaues as quaters.
    activities = defaultdict(list)

    # newcontributors as list in order to append each time a new contributor arrives
    newcontributors = defaultdict(list)
    # oldcontributors as set so that dupplicated can't crawl into the set
    oldcontributors = defaultdict(set)
    
    created, present = get_dates(repo)
    # generating the quaters from `created`  year to `present`  year which are scraped earlier
    for year,quarter,start,end in quarterwise(created,present):
        # add `Qi yyyy`  format as a quater in the quaters list
        quarters.append(r"Q%d %d"%(quarter+1,year))
        # iterating through the contribution types in order to segregate their values into the dict
        for ctype in ctypes:
            # initailizing the counts to zero
            activity =  newcontributor =  0 
            # using the earlier contents dict to check in which quater the data falls
            for item in contents[ctype]:
                # checking if the date of contribtion (commit/issue/pr) created is in between start & end
                if start<=item['created_date'].replace(tzinfo=None)<=end:
                    # it is counted as an activity in that quater
                    activity+=1
                    # checking the author if he is a previous contributor already
                    if item['author'] not in oldcontributors[ctype]:
                        # if not, he is counted as a new contributor
                        newcontributor+=1
                        # and added him to the oldcontributors set
                        oldcontributors[ctype].add(item['author'])
            # total activities are counted and added to the dict as (ctype, value) in list
            activities[ctype].append(activity)
            # newcontributors, either through commit/issue/pr are added to the dict as (ctype, value) in list
            newcontributors[ctype].append(newcontributor)
    # return the data
    return quarters, activities, newcontributors

## Showing the Activity

In [9]:
def show_total_activity(activities):
    """
    Prints the total activity quaterly.
    
    This method prints, from the data, the total 
    activity of the repository based on quaters.
        
    :param activities: get the total activity data 
    """
    print("Quaterwise Total Activity\n")
    # iterating through the activities dict 
    for item in dict(activities):
        # print the total activity quaterly
        print (item, dict(activities)[item])  

In [10]:
def show_new_contributors(newcontributors):
    """
    Prints the number of new contributors quaterly.
    
    This method prints, from the data, the total 
    number of new contributors to the repository 
    based on quaters.
        
    :param newcontributors: get the total
    new contributors data 
    """    
    print("Quaterwise New Contributors Activity\n")
    # iterating through the newcontributors dict 
    for item in dict(newcontributors):
        # print the new activity quaterly
        print (item, dict(newcontributors)[item])  

## Create a CSV to store the Output

In [11]:
def create_output_csv(repo, quaters, activities, newcontributors):
    """
    Creates a CSV file.
    
    This method generates, given the data like repo name
    quaterly details, activites, new contributor details, a csv file.
        
    :param repo: get the repo name
    :param quaters: get the quarters
    :param newcontributors:get the total
    new contributors data 
    :param activities: get the total activity data
    """
    # add headers to the csv file 
    header = ['Quarter','# Commits','# PullRequests','# Issues',
              '# NewCommitters','# NewIssueSubmitters','# NewPRSubmitters' ]
    # opening a new csv to write the data into it.
    with open('%s.csv'%repo, 'w') as file:
        # intilize the writer object
        writer = csv.writer(file)
        # wring the header first
        writer.writerow(header)
        # to map the similar index of multiple containers so that they can be added in single entity i.e, rows
        rows = zip(quarters,activities['commit'],activities['pull_request'],
                   activities['issue'],newcontributors['commit'],
                   newcontributors['pull_request'],newcontributors['issue'])
        # writing all the rows at a time
        writer.writerows(rows)

## Show the Output as a table

In [12]:
def show_as_table(file):
    """
    Creates a table from CSV file data.
    
    This method shows, from the csv file, the data of the csv 
    file as a table.
        
    :param file: obtain the csv data as a file
    
    :returns: a prettified table representing the data
    """
    # to show the output in the form of a table
    # load the csv file into a object
    with open("%s"%file, "r") as csvfile: 
        # using from_csv method from prettytable module
        csvtable = from_csv(csvfile)

    # return the prettified table
    return csvtable

## Summary of the Microtask-1
You need to add few the required repository names in the `repos` list.

> TO-DO
   Make the code free of hardcoded organization names.
   I have mentioned the name of `elastic` organization in the get_dates() funtion as I am trying out with only one repository example.
   We can acheive this by passing repo names as `org/repo` like `elastic/elasticsearch-py` and I need to modify this code accordingly.

In [13]:
# add repo names to analyze
repos = ['elasticsearch-py']

# iterate through the repo list 
for repo in repos:
    # calling the summerize_quaterwise function
    quarters, activities, newcontributors = summerize_quarterwise(repo)
    # calling the show_new_contributors function
    show_new_contributors(newcontributors)
    # calling the show_total_activity function
    show_total_activity(activities) 
    # calling the create_output_csv function
    create_output_csv(repo, quarters, activities, newcontributors)
    # calling the show_as_table function
    print(show_as_table('%s.csv'%repo))

Quaterwise New Contributors Activity

commit [0, 1, 2, 9, 6, 11, 3, 3, 8, 4, 5, 9, 7, 2, 0, 6, 7, 4, 5, 5, 10, 8, 3, 7, 4, 0, 0, 0]
pull_request [0, 0, 2, 9, 16, 15, 8, 11, 12, 7, 6, 12, 10, 3, 5, 14, 12, 6, 8, 9, 10, 9, 4, 8, 8, 0, 0, 0]
issue [0, 0, 5, 16, 32, 25, 25, 30, 28, 31, 25, 38, 46, 20, 36, 42, 36, 33, 31, 25, 37, 38, 27, 26, 13, 0, 0, 0]
Quaterwise Total Activity

commit [0, 78, 154, 94, 126, 33, 30, 38, 45, 37, 27, 80, 28, 29, 21, 34, 27, 28, 31, 57, 44, 26, 10, 37, 4, 0, 0, 0]
pull_request [0, 0, 3, 14, 16, 18, 8, 14, 13, 9, 6, 14, 11, 4, 7, 17, 15, 14, 13, 20, 24, 17, 7, 13, 8, 0, 0, 0]
issue [0, 0, 6, 24, 39, 29, 33, 39, 41, 33, 32, 50, 63, 26, 41, 52, 51, 47, 40, 39, 65, 54, 35, 39, 19, 0, 0, 0]
+---------+-----------+----------------+----------+-----------------+----------------------+-------------------+
| Quarter | # Commits | # PullRequests | # Issues | # NewCommitters | # NewIssueSubmitters | # NewPRSubmitters |
+---------+-----------+----------------+----------+-